### Import Required Libraries and Set Up Environment Variables

In [15]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [3]:
# Set environment variables from the .env in the local environment
load_dotenv()



nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

# Check if the API keys are loaded
print("NYT API Key Loaded: ", "Yes" if nyt_api_key else "No")
print("TMDB API Key Loaded: ", "Yes" if tmdb_api_key else "No")



NYT API Key Loaded:  Yes
TMDB API Key Loaded:  Yes


### Access the New York Times API

In [6]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"{url}api-key={nyt_api_key}&fq={filter_query}&sort={sort}&begin_date={begin_date}&end_date={end_date}&fl={field_list}"

print(query_url)



https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=dEhKwg2K7rIiRWjxKRPVHtK4safPW2gD&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&begin_date=20130101&end_date=20230531&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count


In [7]:
# Create an empty list to store the reviews
reviews_list = []

# Loop through pages 0-19
for page in range(0, 20):
    # Create query with a page number
    query_url = f"{url}api-key={nyt_api_key}&fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&page={page}"

    try:
        # Make a GET request and retrieve the JSON
        response = requests.get(query_url).json()

        # Loop through the reviews["response"]["docs"] and append each review to the list
        reviews = response["response"]["docs"]
        reviews_list.extend(reviews)

        # Print the page that was just retrieved
        print(f"Retrieved page {page}")

    except Exception as e:
        # Print the page number that had no results then break from the loop
        print(f"Error retrieving page {page}: {e}")
        break

    # Add a twelve-second interval between queries to stay within API query limits
    time.sleep(12)

# You can then inspect the first few entries of your reviews_list
print(json.dumps(reviews_list[:5], indent=4))  # Prints the first 5 reviews



Retrieved page 0
Retrieved page 1
Retrieved page 2
Retrieved page 3
Retrieved page 4
Retrieved page 5
Retrieved page 6
Retrieved page 7
Retrieved page 8
Retrieved page 9
Retrieved page 10
Retrieved page 11
Retrieved page 12
Retrieved page 13
Retrieved page 14
Retrieved page 15
Retrieved page 16
Retrieved page 17
Retrieved page 18
Retrieved page 19
[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "su

In [17]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))






[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [18]:
# Convert reviews_list to a Pandas DataFrame
reviews_df = pd.json_normalize(reviews_list)

# Display the first few rows of the DataFrame to verify
print(reviews_df.head())







                                             web_url  \
0  https://www.nytimes.com/2023/05/25/movies/the-...   
1  https://www.nytimes.com/2023/05/04/movies/what...   
2  https://www.nytimes.com/2023/05/04/movies/you-...   
3  https://www.nytimes.com/2023/04/21/movies/a-to...   
4  https://www.nytimes.com/2023/04/20/movies/othe...   

                                             snippet              source  \
0  A gynecologist and her patient form a horrifyi...  The New York Times   
1  Two childhood friends navigate cultural differ...  The New York Times   
2  Religion comes between two girls falling in lo...  The New York Times   
3  Rachael Leigh Cook stars in this bland rom-com...  The New York Times   
4  A radiant Virginie Efira stars as a Parisian t...  The New York Times   

                                            keywords  \
0  [{'name': 'subject', 'value': 'Movies', 'rank'...   
1  [{'name': 'subject', 'value': 'Movies', 'rank'...   
2  [{'name': 'subject', 'value': 'Movi

In [19]:
# Extract the movie title and save to a new column "title"
reviews_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019")] if " Review" in st else None)

# Display the first few rows to verify the new column
print(reviews_df[['headline.main', 'title']].head())





                                       headline.main                   title
0        ‘The Attachment Diaries’ Review: Love, Sick  The Attachment Diaries
1  Review: ‘What’s Love Got to Do With It?’ Proba...                    None
2  ‘You Can Live Forever’ Review: Do You Love Me ...    You Can Live Forever
3  ‘A Tourist’s Guide to Love’ Review: A Wearying...               A Tourist
4     ‘Other People’s Children’ Review: True Romance            Other People


In [20]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"] = reviews_df["keywords"].apply(extract_keywords)
print(reviews_df.head())



                                             web_url  \
0  https://www.nytimes.com/2023/05/25/movies/the-...   
1  https://www.nytimes.com/2023/05/04/movies/what...   
2  https://www.nytimes.com/2023/05/04/movies/you-...   
3  https://www.nytimes.com/2023/04/21/movies/a-to...   
4  https://www.nytimes.com/2023/04/20/movies/othe...   

                                             snippet              source  \
0  A gynecologist and her patient form a horrifyi...  The New York Times   
1  Two childhood friends navigate cultural differ...  The New York Times   
2  Religion comes between two girls falling in lo...  The New York Times   
3  Rachael Leigh Cook stars in this bland rom-com...  The New York Times   
4  A radiant Virginie Efira stars as a Parisian t...  The New York Times   

                                            keywords  \
0  subject: Movies;creative_works: The Attachment...   
1  subject: Movies;persons: Kapur, Shekhar;person...   
2  subject: Movies;creative_works: You

In [22]:
# Extract the movie title and save to a new column "title"
reviews_df['title'] = reviews_df['headline.main'].apply(
    lambda st: st[st.find("\u2018")+1:st.find("\u2019")] if "\u2018" in st and "\u2019" in st and " Review" in st else None
)

# Create a list of movie titles from the "title" column
titles_list = reviews_df['title'].to_list()

# Print the list
print(titles_list)






['The Attachment Diaries', None, 'You Can Live Forever', 'A Tourist', 'Other People', 'One True Loves', 'The Lost Weekend: A Love Story', 'A Thousand and One', 'Your Place or Mine', 'Love in the Time of Fentanyl', 'Pamela, a Love Story', 'In From the Side', 'After Love', 'Alcarràs', 'Nelly & Nadine', 'Lady Chatterley', 'The Sound of Christmas', 'The Inspection', 'Bones and All', 'My Policeman', 'About Fate', 'Waiting for Bojangles', 'I Love My Dad', 'A Love Song', 'Alone Together', 'Art of Love', 'The Wheel', 'Thor: Love and Thunder', 'Both Sides of the Blade', 'Fire of Love', 'Love & Gelato', 'Stay Prayed Up', 'Benediction', 'Dinner in America', 'In a New York Minute', 'Anaïs in Love', 'I Love America', 'See You Then', 'La Mami', 'Love After Love', 'Deep Water', 'Lucy and Desi', 'Cyrano', 'The In Between', 'Book of Love', 'Lingui, the Sacred Bonds', 'The Pink Cloud', 'A Journal for Jordan', 'West Side Story', 'Aulcie', 'Love Is Love Is Love', 'Love Hard', 'Bergman Island', 'Hard Luck 

### Access The Movie Database API

In [23]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key


In [28]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple of 50 requests
request_counter = 0
sleep_after_request = 50

# Set base URL for TMDB API
tmdb_base_url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_details_url = "https://api.themoviedb.org/3/movie/"
tmdb_key_string = "&api_key=" + tmdb_api_key

# Loop through the titles
for title in titles_list:
    # Skip iteration if title is None
    if not title:
        continue

    # Check if we need to sleep before making a request
    if request_counter % sleep_after_request == 0 and request_counter != 0:
        print(f"Sleeping for 12 seconds after {request_counter} requests")
        time.sleep(12)

    # Add 1 to the request counter
    request_counter += 1

    try:
        # Perform a "GET" request for The Movie Database
        movie_response = requests.get(tmdb_base_url + title + tmdb_key_string).json()

        # Get movie id
        movie_id = movie_response["results"][0]["id"]

        # Make a request for the full movie details
        full_movie_response = requests.get(f"{tmdb_details_url}{movie_id}?api_key={tmdb_api_key}").json()

        # Extract the relevant information
        movie_genres = [genre['name'] for genre in full_movie_response["genres"]]
        movie_spoken_languages = [lang['english_name'] for lang in full_movie_response["spoken_languages"]]
        movie_production_countries = [country['name'] for country in full_movie_response["production_countries"]]

        # Add the relevant data to a dictionary and append it to the tmdb_movies_list list
        tmdb_movies_list.append({
            "title": full_movie_response["title"],
            "genre_names": movie_genres,
            "spoken_languages": movie_spoken_languages,
            "production_countries": movie_production_countries
        })

        # Print out the title that was found
        print(f"Movie found: {title}")
    except Exception as e:
        # Print out the title that was not found
        print(f"Movie not found: {title}, Error: {e}")



Movie found: The Attachment Diaries
Movie found: You Can Live Forever
Movie found: A Tourist
Movie found: Other People
Movie found: One True Loves
Movie found: The Lost Weekend: A Love Story
Movie found: A Thousand and One
Movie found: Your Place or Mine
Movie found: Love in the Time of Fentanyl
Movie found: Pamela, a Love Story
Movie found: In From the Side
Movie found: After Love
Movie found: Alcarràs
Movie found: Nelly & Nadine
Movie found: Lady Chatterley
Movie found: The Sound of Christmas
Movie found: The Inspection
Movie found: Bones and All
Movie found: My Policeman
Movie found: About Fate
Movie found: Waiting for Bojangles
Movie found: I Love My Dad
Movie found: A Love Song
Movie found: Alone Together
Movie found: Art of Love
Movie found: The Wheel
Movie found: Thor: Love and Thunder
Movie found: Both Sides of the Blade
Movie found: Fire of Love
Movie found: Love & Gelato
Movie found: Stay Prayed Up
Movie found: Benediction
Movie found: Dinner in America
Movie found: In a New 

In [29]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))




[
    {
        "title": "The Attachment Diaries",
        "genre_names": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
    },
    {
        "title": "You Can Live Forever",
        "genre_names": [
            "Drama",
            "Romance"
        ],
        "spoken_languages": [
            "English",
            "French"
        ],
        "production_countries": [
            "Canada",
            "United States of America"
        ]
    },
    {
        "title": "A Tourist's Guide to Love",
        "genre_names": [
            "Romance",
            "Comedy"
        ],
        "spoken_languages": [
            "English",
            "Vietnamese"
        ],
        "production_countries": [
            "United States of America"
        ]
    },
    {
        "title": "Other People",
        "genr

In [31]:
# Convert the results to a DataFrame
tmdb_movies_df = pd.DataFrame(tmdb_movies_list)
print(tmdb_movies_df.head())



                       title                         genre_names  \
0     The Attachment Diaries  [Drama, Mystery, Thriller, Horror]   
1       You Can Live Forever                    [Drama, Romance]   
2  A Tourist's Guide to Love                   [Romance, Comedy]   
3               Other People                     [Comedy, Drama]   
4             One True Loves            [Romance, Drama, Comedy]   

        spoken_languages                 production_countries  
0              [Spanish]                          [Argentina]  
1      [English, French]   [Canada, United States of America]  
2  [English, Vietnamese]           [United States of America]  
3              [English]           [United States of America]  
4              [English]  [Germany, United States of America]  


### Merge and Clean the Data for Export

In [14]:
# Merge the New York Times reviews and TMDB DataFrames on title


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[French, English]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"[Romance, Drama, Comedy]",en,[English],,Emma and Jesse are living the perfect life tog...,28.872,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,[Documentary],en,[],,May Pang lovingly recounts her life in rock & ...,2.368,97,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,[Drama],en,[English],https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,In Search of Fellini,In Search of Fellini,0,"[Drama, Adventure]",en,[English],,"Lucy, a small-town girl from Ohio, discovers t...",4.103,93,...,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,None,None,None,By Monica Castillo,"[{'firstname': 'Monica', 'middlename': None, '...",None
121,The Last Face,The Last Face,0,[Drama],en,"[English, Portuguese]",,"Miguel, a heroic Spanish doctor, puts himself ...",9.081,130,...,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
122,Lost in Paris,Paris pieds nus,0,[Comedy],fr,[French],,Fiona visits Paris for the first time to assis...,6.798,83,...,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None
123,The Other Half,The Other Half,0,"[Drama, Comedy, Romance]",en,[English],,When English Mark surprises his American bride...,1.058,94,...,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None


In [32]:
# Assuming the columns 'genre_names', 'spoken_languages', and 'production_countries' need fixing
columns_to_fix = ['genre_names', 'spoken_languages', 'production_countries']

# Characters to remove: list brackets and quotation marks
characters_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    tmdb_movies_df[column] = tmdb_movies_df[column].astype(str)

    # Loop through characters to remove
    for char in characters_to_remove:
        tmdb_movies_df[column] = tmdb_movies_df[column].str.replace(char, '', regex=False)

# Display the fixed DataFrame
print(tmdb_movies_df.head())



                       title                       genre_names  \
0     The Attachment Diaries  Drama, Mystery, Thriller, Horror   
1       You Can Live Forever                    Drama, Romance   
2  A Tourist's Guide to Love                   Romance, Comedy   
3               Other People                     Comedy, Drama   
4             One True Loves            Romance, Drama, Comedy   

      spoken_languages               production_countries  
0              Spanish                          Argentina  
1      English, French   Canada, United States of America  
2  English, Vietnamese           United States of America  
3              English           United States of America  
4              English  Germany, United States of America  


In [33]:
# Drop "byline.person" column
reviews_df = reviews_df.drop(columns=["byline.person"])
print(reviews_df.head())



                                             web_url  \
0  https://www.nytimes.com/2023/05/25/movies/the-...   
1  https://www.nytimes.com/2023/05/04/movies/what...   
2  https://www.nytimes.com/2023/05/04/movies/you-...   
3  https://www.nytimes.com/2023/04/21/movies/a-to...   
4  https://www.nytimes.com/2023/04/20/movies/othe...   

                                             snippet              source  \
0  A gynecologist and her patient form a horrifyi...  The New York Times   
1  Two childhood friends navigate cultural differ...  The New York Times   
2  Religion comes between two girls falling in lo...  The New York Times   
3  Rachael Leigh Cook stars in this bland rom-com...  The New York Times   
4  A radiant Virginie Efira stars as a Parisian t...  The New York Times   

                                            keywords  \
0  subject: Movies;creative_works: The Attachment...   
1  subject: Movies;persons: Kapur, Shekhar;person...   
2  subject: Movies;creative_works: You

In [34]:
# Delete duplicate rows and reset index
reviews_df = reviews_df.drop_duplicates(subset=["web_url"])
reviews_df = reviews_df.reset_index(drop=True)
print(reviews_df.head())



                                             web_url  \
0  https://www.nytimes.com/2023/05/25/movies/the-...   
1  https://www.nytimes.com/2023/05/04/movies/what...   
2  https://www.nytimes.com/2023/05/04/movies/you-...   
3  https://www.nytimes.com/2023/04/21/movies/a-to...   
4  https://www.nytimes.com/2023/04/20/movies/othe...   

                                             snippet              source  \
0  A gynecologist and her patient form a horrifyi...  The New York Times   
1  Two childhood friends navigate cultural differ...  The New York Times   
2  Religion comes between two girls falling in lo...  The New York Times   
3  Rachael Leigh Cook stars in this bland rom-com...  The New York Times   
4  A radiant Virginie Efira stars as a Parisian t...  The New York Times   

                                            keywords  \
0  subject: Movies;creative_works: The Attachment...   
1  subject: Movies;persons: Kapur, Shekhar;person...   
2  subject: Movies;creative_works: You

In [35]:
# Export data to CSV without the index
reviews_df.to_csv("output/reviews.csv", index=False)
tmdb_movies_df.to_csv("output/tmdb_movies.csv", index=False)

